### 1.0 Updating Static GTFS Data

### 1.1 Set up

In [1]:
from calendar import calendar
import pandas as pd
import json
from pathlib import Path
from sqlalchemy import create_engine
import geopandas as gpd
import timeit
# from sqlalchemy.orm import Session,sessionmaker
# Using SQLAlchemy to connect to the Database

from sqlalchemy import create_engine,MetaData,event
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import secrets as Config

# from .utils.log_helper import *

from secrets import *
engine = create_engine(Config.URI, echo=False)

Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)

session = Session()
target_schema = "metro_api_dev"
Base = declarative_base(metadata=MetaData(schema=target_schema))

def get_db():
    db = Session()
    try:
        print('Connected to the database')
        yield db
    finally:
        db.close()

list_of_gtfs_static_files = ["stop_times","trips","stops"]
# list_of_gtfs_static_files = ["calendar_dates","calendar","routes","shapes","stop_times","stops","trips"]

debug = True
print('Finished loading libraries')

Finished loading libraries


### 1.2 Loading the data and creating the `stop_times` and `trips` data frames

In [2]:
def check_if_file_exists(gtfs_type,file):
    file_path = "../appdata/gtfs-static/"+gtfs_type+"/" + file + '.txt'
    if Path(file_path).is_file():
        return True
    else:
        return False

def update_gtfs_static_files():
    print('[Debug:{}]'.format(debug))
    process_start = timeit.default_timer()
    global stop_times_df
    global trips_df
    global calendar_dates_df
    global calendar_df
    global stops_df
    for file in list_of_gtfs_static_files:
        if check_if_file_exists("gtfs_bus",file) == True and check_if_file_exists("gtfs_rail",file) == True:
            print('Updating '+file)
            bus_file_path = "../appdata/gtfs-static/gtfs_bus/" + file + '.txt'
            rail_file_path = "../appdata/gtfs-static/gtfs_rail/" + file + '.txt'
            temp_df_bus = pd.read_csv(bus_file_path)
            temp_df_bus['agency_id'] = 'LACMTA'
            temp_df_rail = pd.read_csv(rail_file_path)
            temp_df_rail['agency_id'] = 'LACMTA_Rail'
            if file == "stops":
                stops_df = update_stops_seperately(file)
            
            elif file == "shapes":
                temp_gdf_bus = gpd.GeoDataFrame(temp_df_bus, geometry=gpd.points_from_xy(temp_df_bus.shape_pt_lon, temp_df_bus.shape_pt_lat))   
                temp_gdf_rail = gpd.GeoDataFrame(temp_df_rail, geometry=gpd.points_from_xy(temp_df_rail.shape_pt_lon, temp_df_rail.shape_pt_lat))
                shapes_combined_gdf = gpd.GeoDataFrame(pd.concat([temp_gdf_bus, temp_gdf_rail],ignore_index=True),geometry='geometry')
                shapes_combined_gdf['shape_id_sequence'] = shapes_combined_gdf['shape_id'] +'_' +str(shapes_combined_gdf['shape_pt_sequence'])
                shapes_combined_gdf.crs = {'init': 'epsg:4326'}
                if debug == False:
                    shapes_combined_gdf.to_postgis(file,engine,index=False,if_exists="replace",schema=target_schema)

            else:
                combined_temp_df = pd.concat([temp_df_bus, temp_df_rail])
                if file == "stop_times":
                    stop_times_df = combined_temp_df
                if file == "trips":
                    trips_df = combined_temp_df
                if file == "calendar_dates":
                    calendar_dates_df = combined_temp_df
                if file == "calendar":
                    calendar_df = combined_temp_df
                if debug == False:
                    combined_temp_df.to_sql(file,engine,index=False,if_exists="replace",schema=target_schema)
            process_end = timeit.default_timer()
            print('Updating took {} seconds'.format(process_end - process_start))


def update_stops_seperately(file):
    bus_file_path = "../appdata/gtfs-static/gtfs_bus/" + file + '.txt'
    temp_df_bus = pd.read_csv(bus_file_path)
    # temp_df_bus['geometry'] = [Point(xy) for xy in zip(temp_df_bus.stop_lon, temp_df_bus.stop_lat)] 
    temp_df_bus['agency_id'] = 'LACMTA'
    temp_gdf_bus_stops = gpd.GeoDataFrame(temp_df_bus,geometry=gpd.points_from_xy(temp_df_bus.stop_lon, temp_df_bus.stop_lat))
    temp_gdf_bus_stops.set_crs(epsg=4326, inplace=True)

    rail_file_path = "../appdata/gtfs-static/gtfs_rail/" + file + '.txt'
    temp_df_rail = pd.read_csv(rail_file_path)
    # temp_df_rail['geometry'] = [Point(xy) for xy in zip(temp_df_rail.stop_lon, temp_df_rail.stop_lat)] 
    temp_df_rail['agency_id'] = 'LACMTA_Rail'
    temp_gdf_bus_stops['stop_id'] = temp_gdf_bus_stops['stop_id'].astype('str')
    temp_gdf_bus_stops['stop_code'] = temp_gdf_bus_stops['stop_code'].astype('str')
    temp_gdf_bus_stops['parent_station'] = temp_gdf_bus_stops['parent_station'].astype('str')
    temp_gdf_bus_stops['tpis_name'] = temp_gdf_bus_stops['tpis_name'].astype('str')

    temp_gdf_rail_stops = gpd.GeoDataFrame(temp_df_rail,geometry=gpd.points_from_xy(temp_df_rail.stop_lon, temp_df_rail.stop_lat))
    temp_gdf_rail_stops.set_crs(epsg=4326, inplace=True)
    temp_gdf_rail_stops['stop_id'] = temp_gdf_rail_stops['stop_id'].astype('str')
    temp_gdf_rail_stops['stop_code'] = temp_gdf_rail_stops['stop_code'].astype('str')
    temp_gdf_rail_stops['parent_station'] = temp_gdf_rail_stops['parent_station'].astype('str')
    temp_gdf_rail_stops['tpis_name'] = temp_gdf_rail_stops['tpis_name'].astype('str')
    if debug == False:
        temp_gdf_rail_stops.to_postgis("stops",engine,schema=target_schema,if_exists="replace",index=False)
        temp_gdf_bus_stops.to_postgis("stops",engine,schema=target_schema,if_exists="append",index=False)
    return pd.concat([temp_gdf_bus_stops,temp_gdf_rail_stops])
    
update_gtfs_static_files()

[Debug:True]
Updating stop_times


C:\Users\hikou\AppData\Local\Temp\ipykernel_28564\262173425.py:81: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  update_gtfs_static_files()


Updating took 11.856059100000493 seconds
Updating trips
Updating took 12.272942499999772 seconds
Updating stops
Updating took 12.628031000000192 seconds


### 1.3 Creating the list of trips

In [3]:
def get_day_type_from_service_id(row):
    # print('Getting day type from service id')
    cleaned_row = str(row).lower()
    if 'weekday' in cleaned_row:
        return 'weekday'
    elif 'saturday' in cleaned_row:
        return 'saturday'
    elif 'sunday' in cleaned_row:
        return 'sunday'

def get_day_type_from_trip_id(trip_id):
    # print('Getting day type from trip id')
   this_service_id = trips_df.loc[trips_df['trip_id'] == trip_id, 'service_id'].iloc[0]
   return get_day_type_from_service_id(this_service_id)

def create_list_of_trips(trips,stop_times):
    print('Creating list of trips')
    global trips_list_df
    # stop_times['day_type'] = stop_times['trip_id_event'].map(get_day_type_from_service_id)
    # stop_times['day_type'] = stop_times['day_type'].fillna(stop_times['trip_id'].map(get_day_type_from_trip_id))
    trips_list_df = stop_times.groupby('trip_id')['stop_sequence'].max().sort_values(ascending=False).reset_index()
    trips_list_df = trips_list_df.merge(stop_times[['trip_id','stop_id','stop_sequence','route_code']], on=['trip_id','stop_sequence'])
    summarized_trips_df = trips[["route_id","trip_id","direction_id","service_id","agency_id"]]
    summarized_trips_df['day_type'] = summarized_trips_df['service_id'].map(get_day_type_from_service_id)
    trips_list_df = trips_list_df.merge(summarized_trips_df, on='trip_id').drop_duplicates(subset=['route_id','day_type','direction_id'])
    trips_list_df.to_csv('trips_list_df.csv')


create_list_of_trips(trips_df,stop_times_df)
# print(trips_list_df.head())

Creating list of trips


C:\Users\hikou\AppData\Local\Temp\ipykernel_28564\3645319194.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summarized_trips_df['day_type'] = summarized_trips_df['service_id'].map(get_day_type_from_service_id)


### 1.4 Optional - Check `trip_id` results

Results from this check can be cross-referenced with the **POSTgreSQL database** to see if the `max_sequence` are accurate.

In [4]:
def check_trip_id_times(trip_id):
    print('Checking trip id times')
    this_trips_df = stop_times_df.loc[stop_times_df['trip_id'] == trip_id]
    return this_trips_df


def check_route_id_trips_list(route_id):
    print('Checking route id stop times')
    this_trips_df = trips_list_df.loc[trips_list_df['route_id'] == route_id]
    return this_trips_df

check_trip_id_times('54093732-SoFi_Stadium_Express_1900')
check_trip_id_times('10169001481613-DEC22')

check_route_id_trips_list('169-13167')


Checking trip id times
Checking trip id times
Checking route id stop times


,trip_id,stop_sequence,stop_id,route_code,route_id,direction_id,service_id,day_type
0,10169001481613-DEC22,134,30021,169,169-13167,0,DEC22-D08CAR-1_Weekday,weekday
1,10169001430636-DEC22,134,5505,169,169-13167,1,DEC22-D08CAR-1_Weekday,weekday
3550,70169001391714-DEC22,88,3513,169,169-13167,1,DEC22-D08CAR-3_Sunday,sunday
3579,60169001391918-DEC22,88,3513,169,169-13167,1,DEC22-D08CAR-2_Saturday,saturday
5739,70169001401019-DEC22,83,30021,169,169-13167,0,DEC22-D08CAR-3_Sunday,sunday
5753,60169001401018-DEC22,83,30021,169,169-13167,0,DEC22-D08CAR-2_Saturday,saturday


### 2.0 Getting the stop times data

trips_list_df
- route_id
- trip_id
- stop_sequence

for each row in trips_list_df, apply a function
inputs: route_id, trip_id

function:
get rows from stop_times that match the trip_id
get stop_id column from these rows
add the route_id as a column


resulting dataframe:
| route_id | stop_id |
| -------- | ------- |
| 2 | 100 |
| 2 | 101 |
| 2 | 102 |
| 4 | 104 |

vs

| route_id | stop_id |
| -------- | ------- |
| 2 | [100,101,102] |
| 4 | [104] |


Need to also consider:
- direction
- day of week
- lower priority: active service (calendar)


In [5]:
combined_stop_times_dataframe = pd.DataFrame()

df_to_combine = []

def encode_lat_lon_to_geojson(lat,lon):
    this_geojson = {
        "type":"Feature",
        "geometry":{
            "type":"Point",
            "coordinates": [lon,lat]
        }
    }
    return this_geojson


def get_stops_data_based_on_stop_id(stop_id):
    # print('Getting stops data based on stop id')
    this_stops_df = stops_df.loc[stops_df['stop_id'] == str(stop_id)]
    # print(this_stops_df[['stop_name','stop_lat','stop_lon']])
    # new_object = this_stops_df[['stop_name','stop_lat','stop_lon']].to_dict('records')
    new_object = encode_lat_lon_to_geojson(this_stops_df['stop_lat'].values[0],this_stops_df['stop_lon'].values[0])
    # print('stop_id',stop_id)
    return new_object


def get_stop_times_for_trip_id(this_row):
    this_trips_df = stop_times_df.loc[stop_times_df['trip_id'] == this_row.trip_id]
    this_trips_df['route_id'] = this_row.route_id
    # this_trips_df['service_id'] = this_row.service_id
    this_trips_df['direction_id'] = this_row.direction_id
    this_trips_df['day_type'] = this_row.day_type
    this_trips_df['coordinates'] = this_trips_df.apply(lambda x: get_stops_data_based_on_stop_id(x.stop_id),axis=1)
    this_trips_df['stop_name'] = this_trips_df.apply(lambda x: stops_df.loc[stops_df['stop_id'] == str(x.stop_id)]['stop_name'].values[0],axis=1)
    # simplified_df = this_trips_df[['route_id','stop_id','service_id','day_type','direction_id','stop_name','coordinates']]
    simplified_df = this_trips_df[['route_id','route_code','stop_id','day_type','stop_sequence','direction_id','stop_name','geojson','agency_id']]
    
    df_to_combine.append(simplified_df)
    return simplified_df

# hi_nina_df = get_stop_times_for_trip_id(trips_list_df.iloc[103])
# hi_nina_df.to_csv('hi_nina_df.csv')

#### Running the `get_stop_times_for_trip_id`` function on the entire dataframe
trips_list_df.apply(lambda row: get_stop_times_for_trip_id(row), axis=1)
# trips_list_df.map(get_stop_times_for_trip_id)


0                route_id route_code  stop_id day_type ...
1                route_id route_code  stop_id day_type ...
28               route_id route_code  stop_id day_type ...
37               route_id route_code  stop_id day_type ...
53               route_id route_code  stop_id  day_type...
                               ...                        
38821              route_id route_code  stop_id day_typ...
38972            route_id            route_code  stop_i...
38974            route_id            route_code  stop_i...
39011              route_id route_code  stop_id day_typ...
39275              route_id route_code  stop_id day_typ...
Length: 698, dtype: object

In [6]:
stop_times_by_route_df = pd.concat(df_to_combine)
stop_times_by_route_df.to_csv('stop_times_by_route_df.csv')


In [7]:
#sofi check
sofi_df = stop_times_by_route_df.loc[stop_times_by_route_df['route_id'] == 'SOFI']
sofi_df

,route_id,route_code,stop_id,day_type,stop_sequence,direction_id,stop_name,coordinates
2147877,SOFI,SoFi Stadium Express,30022,None,1,1,Sofi Stadium Transit Center,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2147878,SOFI,SoFi Stadium Express,2319,None,2,1,Hawthorne / Lennox Station,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2147543,SOFI,SoFi Stadium Express,2319,None,1,0,Hawthorne / Lennox Station,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2147544,SOFI,SoFi Stadium Express,30022,None,2,0,Sofi Stadium Transit Center,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


### 2.1 Getting the `stop_times` data

In [8]:
combined_stop_times_array = [] 
import json

def get_stop_times_from_stop_id(this_row):
    # print('Getting stop times for stop id')
    trips_by_route_df = trips_df.loc[trips_df['route_id'] == this_row.route_id]
    
    stop_times_by_trip_df = stop_times_df[stop_times_df['trip_id'].isin(trips_by_route_df['trip_id'])]

    # get the stop times for this stop id
    this_stops_df = stop_times_by_trip_df.loc[stop_times_by_trip_df['stop_id'] == this_row.stop_id]
    this_stops_df = this_stops_df.sort_values(by=['departure_time'],ascending=True)
    # simplified_this_stops_df = simplified_this_stops_df.to_json(orient='records')

    departure_times_array = this_stops_df['departure_time'].values.tolist()
    # to check:
    # print(simplified_this_stops_df)

    # combined_stop_times_array.append(simplified_this_stops_df)
    return departure_times_array

#### for checking purposes
# stop_times_by_route_df['departure_times'] = get_stop_times_from_stop_id(stop_times_by_route_df.iloc[7584])
# get_stop_times_from_stop_id(stop_times_by_route_df.iloc[7585])
# print(stop_times_by_route_df)

# test_df_that_contains_another_df_in_column = pd.DataFrame({'route_id': ['SOFI'], 'departure_df': [this_df]})
# test_df_that_contains_another_df_in_column['test'] = 'hi' 
# test_df_that_contains_another_df_in_column['departure_df'] = get_stop_times_from_stop_id(stop_times_by_route_df.iloc[3])
# stop_times_by_route_df.apply(lambda x: get_stop_times_from_stop_id(x), axis=1).groupby(['route_id','stop_id']).apply(lambda x: pd.concat(x)).to_dict('records').reset_index().to_json(orient='records')
# test_df_that_contains_another_df_in_column


# stop_times_by_route_df.apply(lambda x: get_stop_times_from_stop_id(x), axis=1)
# stop_times_by_route_df.apply(lambda x: get_stop_times_from_stop_id(x['route_id'],x['stop_id']), axis=1)


#### real code here
stop_times_by_route_df['departure_times'] = stop_times_by_route_df.apply(lambda row: get_stop_times_from_stop_id(row),axis=1)

# stop_times_by_route_df.map(get_stop_times_from_stop_id)
# combined_simplified_this_stops_df = pd.concat(combined_stop_times_array)
# combined_simplified_this_stops_df

In [1]:
# final_df = pd.concat(combined_stop_times_array)
from shapely.geometry import Point
def get_lat_long_from_coordinates(geojson):
    this_geojson_geom = geojson['geometry']
    return Point(this_geojson_geom['coordinates'][0], this_geojson_geom['coordinates'][1])

stop_times_by_route_df['route_code'].fillna(stop_times_by_route_df['route_id'], inplace=True)

route_stops_geo_data_frame = gpd.GeoDataFrame(stop_times_by_route_df, geometry=stop_times_by_route_df.apply(lambda x: get_lat_long_from_coordinates(x.coordinates),axis=1))
route_stops_geo_data_frame.set_crs(epsg=4326, inplace=True)



# if you want to pre-process the data before saving it to the database, do it here! :)
if debug == False:
    # save to database
    route_stops_geo_data_frame.to_postgis('route_stops',engine,index=False,if_exists="replace",schema=target_schema)
    # stop_times_by_route_df.to_csv('final_df.csv')
# route_stops_geo_data_frame.to_postgis('route_stops',engine,index=False,if_exists="replace",schema=target_schema)
# stop_times_by_route_df.to_csv('final_df.csv')

NameError: name 'stop_times_by_route_df' is not defined

In [ ]:
route_stops_geo_data_frame
route_stops_geo_data_frame['route_code'].astype('str')

def get_individual_route_data(target_data):
       initial_json = (target_data.groupby(['direction_id','day_type'])
       .apply(lambda x: x.to_dict('records'))
       .reset_index()
       .rename(columns={0:'data'})
       .groupby('direction_id')['day_type','data']
       .apply(lambda x:x.set_index('day_type')['data'].to_dict())
       .to_json(orient='index',indent=4)
       )
       cleaned_json = [json.loads(initial_json)]
       # initial_json.to_dict()
       return cleaned_json


def get_departure_times(route):
       target_data = route_stops_geo_data_frame.loc[route_stops_geo_data_frame['route_code'].astype('str') == route]
       if target_data.shape[0] > 0:
              payload = []
       # payload.append(target_data.apply(lambda x: get_individual_route_data(x),axis=1))
              payload.append(get_individual_route_data(target_data))
       
              final_payload = [pd.json_normalize(payload).assign(route_code=route).to_dict('records')]
       # final_payload = [pd.json_normalize(payload).assign(route_code=route).to_dict('records')]


       # initial_json = (target_data.groupby(['direction_id','day_type'])['departure_times','stop_sequence','stop_name','stop_id','coordinates']
       # .apply(lambda x: x.to_dict('r'))
       # .reset_index(name='data')
       # .groupby('direction_id')['day_type','data']
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .to_json(orient='index',indent=4)
       # )
       # cleaned_json = json.loads(initial_json)
       # initial_json.to_dict()
              return final_payload
       else:
              return None
final_df = pd.DataFrame()
route_code_df = pd.DataFrame({'route_code':route_stops_geo_data_frame['route_code'].unique().astype('str')})
route_id_df = pd.DataFrame({'route_id':route_stops_geo_data_frame['route_id'].unique().astype('str')})
final_df = pd.concat([route_code_df,route_id_df],ignore_index=True, axis=1)
final_df.columns = ['route_code','route_id']
# final_df['route_id'].fillna(final_df['route_code'], inplace=True)


# final_df['agency_id'] = route_stops_geo_data_frame[''].unique()
final_df['payload'] = final_df['route_code'].apply(lambda x: get_departure_times(x))
print(final_df)
# final_df
# final_df.to_postgis('route_stops_grouped',engine,index=False,if_exists="replace",schema=target_schema)

<!-- for each `route_id`, we have a list of `trip_ids` -->

`trips_list_df `-> max_sequence, route_id, trip_id

`trips_df` -> trips table

`stop_times_df` -> stop_times table

`stop_times_by_route_df` -> stop_times based on `trip_id` with max_sequence for each `route_id`: route_id, stop_id

apply this function to each row in `stop_times_by_route_df`:
(given a `route_id` and `stop_id`)

a) new `trips_by_route_df` = filter `trips_df` by `route_id`

new `stop_times_by_trip_df` = filter `stop_times_df` by `trip_ids` within `trips_by_route_df`

new `results_df` = `stop_times_by_trip_df` columns: `departure_times`, `stop_id`



### Updated example: 1/26/2023
```json
{
  route_id: '',
  direction_0: 
  {
    weekday:
    [
      {
        stop_id: '',
        stop_name: '',
        stop_sequence: '',
        coordinate: '',
        departure_times: 
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ]
      },{
        stop_id: '',
        stop_name: '',
        stop_sequence: '',
        coordinate: '',
        departure_times: 
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ]
      }
    ],
    saturday: 
    [
      {}
    ],
    sunday:
    [
      {}
    ]
  },
  direction_1: 
  {
    weekday:
    [
      {}
    ],
    saturday: 
    [
      {}
    ],
    sunday:
    [
      {}
    ]
  }
}
```

``` js
[
  {
    route_id: `801`,
    stop_id: `80214`,
    stop_name: 'Union Station - Metro Red & Purple Lines',
    coordinate: .....,
    stop_sequence: {
      direction_0: {
        weekday: 1,
        saturday: 1,
        sunday: 1
      },
      direction_1: {
        weekday: 14
        saturday: 14
        sunday: 14
      }
    }
    stop_times: {
      direction_0: {
        weekday: [
          '05:01:00',
          ...
        ],
        saturday: [
          '05:01:00',
          ...
        ],
        sunday: [
          '05:01:00',
          ...
        ]
      },
      direction_1: {
        weekday: [
          '05:01:00',
          ...
        ],
        saturday: [
          '05:01:00',
          ...
        ],
        sunday: [
          '05:01:00',
          ...
        ]
      }
    }
  },
  {
   ...
  }
]
```